In [189]:
import pandas as pd

In [196]:
data = pd.read_csv('tum_edges_list.csv', header=0, usecols=[0,1,2])

In [191]:
class Node:
    def __init__(self, label):
        self.label = label

class Edge:
    def __init__(self, to_node, length):
        self.to_node = to_node
        self.length = length


class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = dict()

    def add_node(self, node):
        self.nodes.add(node)

    def add_edge(self, from_node, to_node, length):
        edge = Edge(to_node, length)
        if from_node.label in self.edges:
            from_node_edges = self.edges[from_node.label]
        else:
            self.edges[from_node.label] = dict()
            from_node_edges = self.edges[from_node.label]
        from_node_edges[to_node.label] = edge


def min_dist(q, dist):
    """
    Returns the node with the smallest distance in q.
    Implemented to keep the main algorithm clean.
    """
    min_node = None
    for node in q:
        if min_node == None:
            min_node = node
        elif dist[node] < dist[min_node]:
            min_node = node

    return min_node

INFINITY = float('inf')

def dijkstra(graph, source):
    q = set()
    dist = {}
    prev = {}

    for v in graph.nodes:       # initialization
        dist[v] = INFINITY      # unknown distance from source to v
        prev[v] = INFINITY      # previous node in optimal path from source
        q.add(v)                # all nodes initially in q (unvisited nodes)

    # distance from source to source
    dist[source] = 0

    while q:
        # node with the least distance selected first
        u = min_dist(q, dist)

        q.remove(u)

        if u.label in graph.edges:
            for _, v in graph.edges[u.label].items():
                alt = dist[u] + v.length
                if alt < dist[v.to_node]:
                    # a shorter path to v has been found
                    dist[v.to_node] = alt
                    prev[v.to_node] = u

    return dist, prev


def to_array(prev, from_node):
    """Creates an ordered list of labels as a route."""
    previous_node = prev[from_node]
    route = [from_node.label]
    while previous_node != INFINITY:
        route.append(previous_node.label)
        temp = previous_node
        previous_node = prev[temp]

    route.reverse()
    return route

In [197]:
graph = Graph()
nodes = sorted(pd.unique(data[['from_node', 'to_node']].values.ravel()))

In [211]:
for x in range(0, len(nodes)):    
    globals()[nodes[x]] = Node(nodes[x])
    graph.add_node(globals()[nodes[x]])

In [210]:
%%timeit
for row in range(0, len(data)):
    to_node = globals()[data['to_node'][row]]
    from_node = globals()[data['from_node'][row]]
    graph.add_edge(from_node, to_node, data['weight'][row])

3.17 ms ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [208]:
dist, prev = dijkstra(graph, table_empty)

85.2 µs ± 458 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [207]:
print("The quickest path from {} to {} is [{}] with a distance of {}".format(
    table_empty.label,
    pcstn.label,
    " -> ".join(to_array(prev, pcstn)),
    str(round(dist[pcstn], 1))
    )
)

The quickest path from table_empty to pcstn is [table_empty -> silverware -> pcs -> pcstn] with a distance of 18.8
